# CAMEL AI Examples

We will demonstrate two examples of using CAMEL AI with AgentOps.

First let's install the dependencies for AgentOps/CamelAI

In [ ]:
%pip install agentops
%pip install camel-ai[all]

First we import the necessary libraries

In [ ]:
import agentops
import os
from dotenv import load_dotenv

# Camel imports
from camel.agents import ChatAgent
from camel.messages import BaseMessage
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.toolkits import SearchToolkit

Then we load our API keys

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or "<your openai key here>"
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your agentops key here>"

Now we initialize our AgentOps client!

In [ ]:
agentops.init(api_key=AGENTOPS_API_KEY, default_tags=["camel", "examples", "single-agent", "multi-agent"])

## Simple Assistant Example

We will use CamelAI to simulate talking to an assistant. We will replicate Blade Wolf from Metal Gear Rising: Revengeance. We will also use a CSV sheet to quickshot train our LLM.

And now we setup our AI based off of the information we give it through the information we give it below

In [4]:
sys_msg = BaseMessage.make_assistant_message(
    role_name='Tools calling opertor', content='You are a helpful assistant named Bladewolf, based off of the Metal Gear Rising Series.'
)

We are going to read a CSV file to train our LLM.

In this case, we know the first row in our CSV contains assistant messages while the right side contains user messages. 

In [5]:
# Import the csv class
import csv

# Let's open the file in Python
with open('bladewolf_training.csv', 'r') as file:
    csv_reader = csv.reader(file)

    # Be sure to skip the first row (Label)
    next(csv_reader)
    
    # Iterate over each row in the csv file
    for row in csv_reader:
        # Ensure each row has exactly 2 columns before proceeding
        if len(row) == 2:
            BaseMessage.make_assistant_message(role_name="assistant", content=row[0])  # Add assistant context
            BaseMessage.make_user_message(role_name="user", content=row[1])  # Add user context


And we set the basics required to get this running; essentially the model tools, the model itself (Based on the API) and the agent details.

In [ ]:
# Set model config
tools = [*SearchToolkit().get_tools()]

#This means we are using GPT_4O_MINI
model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_4O_MINI,
)

# Set agent
camel_agent = ChatAgent(
    system_message=sys_msg,
    model=model,
    tools=tools,
)

Everything is set up! Now we can run our agent and see the magic happen!

In [ ]:
# Define a user message
usr_msg = 'I need a breifing on the T-84 Metal Gear, code named Sahelanthropus and how we can beat it. Introduce yourself as well.'

# Get response information
response = camel_agent.step(usr_msg)
print(response.msg.content)

Awesome! We can see the response from our agent and we can also see the session in the AgentOps dashboard by clicking on the session URL provided above.

## Multi-Agent Chat Example

In this example, we'll create two agents: a Python expert who writes code and a code reviewer who provides feedback.

In [8]:
# Initialize two CAMEL agents
assistant = ChatAgent(
    model=ModelFactory.create(
        model_platform=ModelPlatformType.OPENAI,
        model_type=ModelType.GPT_4O_MINI,
    ),
    system_message="You are a Python expert who helps write clean, efficient code."
)

reviewer = ChatAgent(
    model=ModelFactory.create(
        model_platform=ModelPlatformType.OPENAI,
        model_type=ModelType.GPT_4O_MINI,
    ),
    system_message="You are a code reviewer who focuses on code quality and best practices."
)

In [ ]:
# Start a conversation about code optimization
message = BaseMessage(
    role_name="Human",
    role_type="human",
    meta_dict={"timestamp": "2024-01-01T00:00:01"},
    content="I need help optimizing this Python function for calculating prime numbers:\ndef is_prime(n):\n    for i in range(2, n):\n        if n % i == 0:\n            return False\n    return True"
)

# Get assistant's response
assistant_response = assistant.step(message)
print("Assistant's suggestion:", assistant_response.msgs[0].content)

# Get reviewer's feedback
reviewer_message = BaseMessage(
    role_name="Human",
    role_type="human",
    meta_dict={"timestamp": "2024-01-01T00:00:02"},
    content=f"Please review this code suggestion:\n{assistant_response.msgs[0].content}"
)
reviewer_response = reviewer.step(reviewer_message)
print("\nReviewer's feedback:", reviewer_response.msgs[0].content)

# Implement reviewer's suggestions
improvement_message = BaseMessage(
    role_name="Human",
    role_type="human",
    meta_dict={"timestamp": "2024-01-01T00:00:03"},
    content=f"Please improve the code based on this feedback:\n{reviewer_response.msgs[0].content}"
)
final_response = assistant.step(improvement_message)
print("\nFinal improved code:", final_response.msgs[0].content)

You can see on the AgentOps dashboard that we have a multi-agent chat session. Since we did not specify a session name, the session name will be the default name of the class i.e. `ChatAgent`.

Now we will end the session with a success message. We can also end the session with a failure or intdeterminate status. By default, the session will be marked as indeterminate.

In [ ]:
agentops.end_session("Success")